<a href="https://colab.research.google.com/github/vladgap/Various/blob/main/MLN-281121_Regres_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
# import pandas as pd
from sklearn import preprocessing

# StandardScaler

In [2]:
x=[
[2,1,0],
[-1,0,1],
[0,0,0],
[1,1,0],
[-1,-1,-1],
[-1,1,-1],
]
# x=np.array(x)
scaler = preprocessing.StandardScaler().fit(x)
print(scaler.mean_, scaler.scale_)
x_scaled = scaler.transform(x)
print (x_scaled)
x_back=scaler.inverse_transform(x_scaled)
print (x_back)
scaler.inverse_transform([[0,0,0]])
# help(scaler)

[ 0.          0.33333333 -0.16666667] [1.15470054 0.74535599 0.68718427]
[[ 1.73205081  0.89442719  0.24253563]
 [-0.8660254  -0.4472136   1.69774938]
 [ 0.         -0.4472136   0.24253563]
 [ 0.8660254   0.89442719  0.24253563]
 [-0.8660254  -1.78885438 -1.21267813]
 [-0.8660254   0.89442719 -1.21267813]]
[[ 2.  1.  0.]
 [-1.  0.  1.]
 [ 0.  0.  0.]
 [ 1.  1.  0.]
 [-1. -1. -1.]
 [-1.  1. -1.]]


array([[ 0.        ,  0.33333333, -0.16666667]])

# Single Neuron

In [57]:
class Neuron:
    """A single neuron with an activation function.
       Attributes:
          inputs: The number of inputs in the perceptron, not counting the bias.
          bias:   The bias term. By defaul it's 1.0.
          activ:  The activation function: linear (default), relu, sigmoid."""

    def __init__(self, inputs, bias = 1.0, activ = 'linear'):
        """Return a new Perceptron object with the specified number of inputs (+1 for the bias) and random initial weights.""" 
        self.weights = (np.random.rand(inputs+1) * 2) - 1 
        self.bias = bias
        self.activ = activ

    def run(self, x):
        """Run the perceptron. x is a python list with the input values."""
        sum = np.dot(np.append(x,self.bias),self.weights)
        if self.activ == 'linear':
          return sum
        if self.activ == 'sigmoid':
          return self.sigmoid(sum)
        if self.activ == 'relu':
          return self.relu(sum)
        if self.activ == 'mrelu':
          return self.mrelu(sum)

    def set_weights(self, w_init):
        """Overrides the np.random.rand() weights and the bias weight."""
        # w_init is a list of floats. Organize it as you'd like.
        self.weights=np.array(w_init, dtype='double')

    def set_activ(self, activ, param=0):
        """Overrides the 'linear' activation function."""
        self.activ = activ
        self.param = param

    def sigmoid(self, x):
        # return the output of the sigmoid function applied to x
        return 1/(1+np.exp(-x))
    
    def relu(self, x):
        # return the output of the relu function applied to x
        if x >= 0:
          return x
        return 0

    def mrelu(self, x):
        # return the output of the modified relu function applied to x
        if x >= 0:
          return x
        return self.param*x

In [4]:
neu=Neuron(inputs=2)
neu.set_weights([10,10,-15]) 
neu.set_activ('mrelu', param=0.01)
neu.run([1,0])

-0.05

## AND gate

In [5]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([10,10,-15]) #AND gate

print("AND Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))


AND Gate:
0 0 = 0.0000003059
0 1 = 0.0066928509
1 0 = 0.0066928509
1 1 = 0.9933071491


## OR gate

In [6]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([10,10,-5]) #OR gate

print("OR Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))


OR Gate:
0 0 = 0.0066928509
0 1 = 0.9933071491
1 0 = 0.9933071491
1 1 = 0.9999996941


## NAND gate

In [7]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([-10,-10,15]) #NAND gate

print("NAND Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))

NAND Gate:
0 0 = 0.9999996941
0 1 = 0.9933071491
1 0 = 0.9933071491
1 1 = 0.0066928509


# Multilayer neuron

In [8]:
class MultiLayerNeuron:     
    """A multilayer neuron class that uses the Neuron class above.
       Builds a list of neurons with the specific activation function.
       The activation function may be modified later using the set_activ method.
       For example: mln.network[layer][neuron].set_activ('linear').
       Attributes:
          layers:  A list with the number of neurons per layer. Including the input (first) and the output (last) layers.
          bias:    The bias term. The same bias is used for all neurons.
          eta:     The learning rate.
          activ:   The activation function: linear (default), relu, sigmoid."""

    def __init__(self, layers, bias = 1.0, eta = 0.5, activ='linear'):
        """Return a new MLP object with the specified parameters.""" 
        self.layers = np.array(layers,dtype=object)
        self.bias = bias
        self.eta = eta
        self.network = [] # The list of lists of neurons (perceptrons).
        self.values = []  # The list of lists of neurons' (perceptrons') output values.
        self.d = []       # The list of lists of error terms (lowercase deltas)
        self.activ = activ

        # 2 nested loops to create neurons layer by layer
        for i in range(len(self.layers)): # outer loop iterates on each layer
            self.values.append([]) #The new list of values will be filled with zeros, for every neuron in the layer. 
            self.values[i] = [0.0 for j in range(self.layers[i])]
            self.d.append([])
            self.d[i] = [0.0 for j in range(self.layers[i])]                        
            self.network.append([])
            if i > 0:      #network[0] is the input layer, so it has no neurons
                for j in range(self.layers[i]): # inner loop iterates on each neuron in a layer
                    neur=Neuron(inputs = self.layers[i-1], bias = self.bias, activ = self.activ) # 
                    self.network[i].append(neur) # adding j perceptrons
        
        self.network = np.array([np.array(x) for x in self.network],dtype=object) #transforms list of lists to numpy array
        self.values = np.array([np.array(x) for x in self.values],dtype=object)
        self.d = np.array([np.array(x) for x in self.d],dtype=object)

    def set_weights(self, w_init): # set_weights of the MultiLayer class
        """Set the weights. 
           w_init is a list of lists with the weights for all but the input layer."""
        for i in range(len(w_init)):
            for j in range(len(w_init[i])):
                self.network[i+1][j].set_weights(w_init[i][j]) # set_weights for each perceptron i

    def set_activ(self, activ, param=0):
        """Set the activation function to every neurons."""
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                self.network[i][j].set_activ(activ, param) # set_activ for each neuron
        self.param=param
    
    def set_output_activ(self, activ, param=0):
        """Set the activation function to the last (output) neurons."""
        i = len(self.network)-1
        for j in range(self.layers[i]):
            self.network[i][j].set_activ(activ, param) 

    def printWeights(self):
        """Displays a summary of weights and activation functions per layer and neuron."""
        print()
        print('Layer 0 is the Input Layer')
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                print("Layer",i,"Neuron",j,":",self.network[i][j].weights,self.network[i][j].activ)
        print()

    def run(self, x):
        """Feed a sample x into the MultiLayer Neuron."""
        x = np.array(x,dtype=object)
        self.values[0] = x
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):  
                self.values[i][j] = self.network[i][j].run(self.values[i-1]) #runs preceptrons with the previous outputs
        return self.values[-1]

    def bp_classif(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorithm - Gradient Descent.
        Uses the derivative of the sigmoid function."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)
        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        # STEP 2: Calculate the MSE
        error = 2*(y - outputs) # A list of outputs
        MSE = sum( error ** 2) / self.layers[-1] 
        # ∂MSE/∂weight=∂MSE/∂output*∂output/∂weight
        # STEP 3: Calculate the OUTPUT error terms
        # ∂MSE/∂output -- depends on neuron's activation function
        self.d[-1] = outputs * (1 - outputs) * (error) # derivative of the SIGMOID function 
        # STEP 4: Calculate the error term of EACH UNIT on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k]               
                self.d[i][h] = self.values[i][h] * (1-self.values[i][h]) * fwd_error # derivative of the SIGMOID function
        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)): # runs on layers
            for j in range(self.layers[i]): # runs on neurons
                for k in range(self.layers[i-1]+1): # runs on inputs. +1 for bias
                    if k==self.layers[i-1]:
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        delta = self.eta * self.d[i][j] * self.values[i-1][k] # applying the delta rule
                    self.network[i][j].weights[k] += delta
        return MSE

    def sigmoid(self, x):
        # return the output of the sigmoid function applied to x
        return 1/(1+np.exp(-x))

    def deriv(self, value, i, j=0):
        if self.network[i][j].activ == 'linear':
          # print ('lin')
          return 1
        if self.network[i][j].activ == 'sigmoid':
          # print ('sig')
          return self.sigmoid(value)*(1-self.sigmoid(value))
        if self.network[i][j].activ == 'relu':
          if value > 0:
            # print ('re>')
            return 1
          else:
            # print ('re<')
            return 0
        if self.network[i][j].activ == 'mrelu':
          if value > 0:
            return 1
          else:
            return self.param

    def bp_regres(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorithm - Gradient Descent.
        Uses the derivative according each neuron's activation function."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)
        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        # STEP 2: Calculate the MSE
        error = 2*(y - outputs) # A list of outputs
        MSE = sum( error ** 2) / self.layers[-1] 
        # ∂MSE/∂weight=∂MSE/∂output*∂output/∂weight
        # STEP 3: Calculate the OUTPUT error terms
        # ∂MSE/∂output -- depends on neuron's activation function
        for j in range (len(outputs)):
            self.d[-1][j] = self.deriv(outputs[j], len(self.network)-1) * error
        # STEP 4: Calculate the error term of EACH UNIT on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k] 
                self.d[i][h] = self.deriv(self.values[i][h], i, h) * fwd_error
        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)): # runs on layers
            for j in range(self.layers[i]): # runs on neurons
                for k in range(self.layers[i-1]+1): # runs on inputs. +1 for bias
                    # output=sum(weight*value)+bias*bias_weight
                    if k==self.layers[i-1]:
                        # ∂output/∂bias_weight=bias
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        # ∂output/∂weight=value
                        delta = self.eta * self.d[i][j] * self.values[i-1][k] 
                    self.network[i][j].weights[k] += delta # applying the delta rule
        return MSE

## XOR gate=(OR+NAND)+AND

In [60]:
#test code
mln1 = MultiLayerNeuron(layers=[2,2,1])  #mln1
mln1.set_weights([[[-10,-10,15],[15,15,-10]],[[10,10,-15]]])
mln1.set_activ('sigmoid') #linear is by default

mln1.printWeights()
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln1.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln1.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln1.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln1.run([1,1])[0]))


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-10. -10.  15.] sigmoid
Layer 1 Neuron 1 : [ 15.  15. -10.] sigmoid
Layer 2 Neuron 0 : [ 10.  10. -15.] sigmoid

XOR Gate:
0 0 = 0.0066958493
0 1 = 0.9923558642
1 0 = 0.9923558642
1 1 = 0.0071528098


## Training 

In [10]:
mln2 = MultiLayerNeuron(layers=[2,2,1])
mln2.set_activ('sigmoid') #linear is by default
print("\nTraining Neural Network as an XOR Gate...\n")
for i in range(2000):
    MSE = 0.0
    MSE += mln2.bp_classif([0,0],[0])
    MSE += mln2.bp_classif([0,1],[1])
    MSE += mln2.bp_classif([1,0],[1])
    MSE += mln2.bp_classif([1,1],[0])
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln2.printWeights()
    
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln2.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln2.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln2.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln2.run([1,1])[0]))


Training Neural Network as an XOR Gate...

1.10112588256286
0.8698327571664068
0.15342403917644165
0.03204574468934282
0.01622043620258888
0.010601124611834122
0.007794366221841156
0.0061298225638090065
0.005034807968220626
0.004262632339855672

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-4.48897796 -4.48969148  6.66793715] sigmoid
Layer 1 Neuron 1 : [-5.97885462 -5.9967431   2.37374255] sigmoid
Layer 2 Neuron 0 : [ 8.71364178 -8.97717984 -4.07664372] sigmoid

XOR Gate:
0 0 = 0.0269509971
0 1 = 0.9711720591
1 0 = 0.9710729343
1 1 = 0.0358830044


# Regres

In [31]:
class Regres:
    def __init__(self, layers, bias=1.0, eta=0.5, ):
        self.layers = layers
        self.bias = bias
        self.eta = eta
        self.regres_network = MultiLayerNeuron(layers=layers, bias=bias, eta=eta)

    def set_weights(self, w_init):
        self.regres_network.set_weights(w_init)

    def set_hidden_activ(self, activ, param=0):
        self.regres_network.set_activ(activ, param=0)
        self.regres_network.set_output_activ('linear')

    def fit(self, X, y, epochs):
        self.epochs=epochs
        self.weight_history=[]
        self.MSE_history=[]
        for i in range(self.epochs):
            weight_epoch=[]
            MSE = 0.0
            for j in range (len(X)):
                MSE +=  self.regres_network.bp_regres(X[j],[y[j]])
            MSE = MSE / len(X)
            for m in range(1,len(self.regres_network.network)):
                for n in range(self.layers[m]):
                    weight_epoch.append(self.regres_network.network[m][n].weights)
            self.weight_history.append(weight_epoch)
            self.MSE_history.append(MSE)
        print ("""Model fitted.
self.weight_history - list of lists of weights propagation
self.MSE_history - list of MSEs propagation""")

    def printWeights(self):
        self.regres_network.printWeights()

    def run(self, x):
        return self.regres_network.run(x)

    def predict(self, x):
        y=[]
        for i in range(len(x)):
            y.append(self.run(x[i])[0])
        return y

# Regres_scale

In [12]:
# nuzhno podumat' kak vesa obratno perevesti. mozhet inverse_transform
class Regres_scale:
    def __init__(self, layers, bias=1.0, eta=0.5, ):
        self.layers = layers
        self.bias = bias
        self.eta = eta
        # self.epochs = epochs      
        self.regres_network = MultiLayerNeuron(layers=layers, bias=bias, eta=eta)

    def set_weights(self, w_init):
        self.regres_network.set_weights(w_init)

    def set_hidden_activ(self, activ):
        self.regres_network.set_activ(activ)
        self.regres_network.set_output_activ('linear')

    def fit(self, x, y, epochs):
        self.scaler = preprocessing.StandardScaler().fit(x)
        x_scaled=self.scaler.transform(x)
        self.epochs=epochs
        for i in range(epochs):
            MSE = 0.0
            for j in range (len(x)):
                MSE +=  self.regres_network.bp_regres(x_scaled[j],[y[j]])
            MSE = MSE / len(x)
            # dobavit' istoriyu kak v Regres

    def printWeights(self):
        self.regres_network.printWeights()

    def run(self, x):
        return self.regres_network.run(x)

    def predict(self, x):
        y=[]
        for i in range(len(x)):
            y.append(self.run(x[i])[0])
        return y

    def get_standard_params(self, x):
        x2=np.array(x)
        mean=x2.mean(axis=0)
        stdev=x2.std(axis=0)
        return mean, stdev

# Example 1 -- weights

In [13]:
print ("1 1 =",mln1.run([1,1]))
print (mln1.network[1][0].weights) # network is list of lists of perceptrons. Each has attribute "weights"

1 1 = [0.00715281]
[-10 -10  15]


In [14]:
mln1=MultiLayerNeuron(layers=[2,2,1])
mln1.set_activ('relu')
mln1.printWeights()

mln1.set_output_activ('sigmoid') # setting the output activ func
mln1.printWeights()

mln1.network[1][0].set_activ('linear') # changing specific activ func
mln1.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.44377905 -0.13477171  0.67567793] relu
Layer 1 Neuron 1 : [ 0.52571212 -0.53777991  0.61644181] relu
Layer 2 Neuron 0 : [-0.2953755  -0.71534293  0.22458393] relu


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.44377905 -0.13477171  0.67567793] relu
Layer 1 Neuron 1 : [ 0.52571212 -0.53777991  0.61644181] relu
Layer 2 Neuron 0 : [-0.2953755  -0.71534293  0.22458393] sigmoid


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.44377905 -0.13477171  0.67567793] linear
Layer 1 Neuron 1 : [ 0.52571212 -0.53777991  0.61644181] relu
Layer 2 Neuron 0 : [-0.2953755  -0.71534293  0.22458393] sigmoid



# Example 3 -- linear activs, bp_regres. מתכנס למשקלים טוב מאוד 

In [59]:
mln3 = MultiLayerNeuron(layers=[3,1], eta=0.3)
# mln3.set_weights([[[5.,-0.9,2.,0.]]])
mln3.set_weights([[[0,0,0,0]]])
mln3.printWeights()
print("\nTraining Neural Network...\n")
for i in range(400):
    MSE = 0.0
    MSE += mln3.bp_regres([0.7759,	0.1104,	0.9977,],5.764286995)
    MSE += mln3.bp_regres([0.9692,	0.6961,	0.8483,],5.84646758)
    MSE += mln3.bp_regres([0.0265,	0.399,	0.5375,],0.808633075)
    MSE += mln3.bp_regres([0.7694,	0.5051,	0.2542,],3.850298589)
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln3.printWeights()




Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0. 0. 0. 0.] linear


Training Neural Network...

54.560573414122615
9.652321993077698e-13

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 4.99973369e+00 -9.99625006e-01  1.99991681e+00  3.52253294e-05] linear



# Example 4 -- sigmoid, bp_regres. מקרב רגרסיה עם סיגמויד. עובד טוב

In [62]:
# generating output values
mln4 = MultiLayerNeuron(layers=[3,2,1])
mln4.set_activ('sigmoid')
mln4.set_output_activ('linear')
mln4.set_weights([[[0,1,-2,1],[1,-2,0,1]],[[-1,2,3]]])
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('original values:',y4)
print ()

# training model and predicting values
mln4 = MultiLayerNeuron(layers=[3,2,1], eta=0.1)
mln4.set_activ('sigmoid')
mln4.set_output_activ('linear')

for i in range(1000):
    MSE = 0.0
    MSE += mln4.bp_regres([2,1,0],y4[0])
    MSE += mln4.bp_regres([-1,0,1],y4[1])
    MSE += mln4.bp_regres([0,0,0],y4[2])
    MSE += mln4.bp_regres([1,1,0],y4[3])
    MSE += mln4.bp_regres([-1,-1,-1],y4[4])
    MSE += mln4.bp_regres([-1,1,-1],y4[5])
    MSE = MSE / 6
    if(i%200 == 0):
        print ('MSE=',MSE)
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('predicted values:',y4)
print ()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.  1. -2.  1.] sigmoid
Layer 1 Neuron 1 : [ 1. -2.  0.  1.] sigmoid
Layer 2 Neuron 0 : [-1.  2.  3.] linear



'original values:'

[3.5813200792821274,
 3.731058578630005,
 3.731058578630005,
 3.119202922022118,
 3.880797077977882,
 2.2563920540063265]


MSE= 20.787125050357307
MSE= 0.021306488911559635
MSE= 0.01035468792406984
MSE= 0.003917334748799554
MSE= 0.0019310623252030451

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.85818005 -0.8968281   0.11073781  0.90633917] sigmoid
Layer 1 Neuron 1 : [ 1.15613088 -3.43124786  1.88436541  0.20321723] sigmoid
Layer 2 Neuron 0 : [1.5965536  1.46266457 1.81693837] linear



'predicted values:'

[3.590286388330205,
 3.7277752018659402,
 3.75947166704797,
 3.104946232069896,
 3.8724982554383396,
 2.261945428849782]

# Example 5 -- relu-sigmoid, bp_regres. מתכנס למשקלים לא משהו

In [69]:
mln4 = MultiLayerNeuron(layers=[3,2,1,1]) # generating MLN with random weights
mln4.set_activ('relu')
mln4.network[2][0].set_activ('sigmoid') # changing specific activ func
mln4.set_output_activ('linear')
mln4.set_weights([[[0,1,-2,1],[1,-2,0,1]],[[1,0,1]],[[-1,2]]]) # setting weights
mln4.printWeights()

y4=[] # generating output values
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('original values:',y4)
print ()

# training model and predicting values
mln4 = MultiLayerNeuron(layers=[3,2,1,1], eta=0.3) # generating MLN with random weights
mln4.set_activ('relu')
mln4.network[2][0].set_activ('sigmoid') # changing specific activ func
mln4.set_output_activ('linear')

for i in range(400):
    MSE = 0.0
    MSE += mln4.bp_regres([2,1,0],y4[0])
    MSE += mln4.bp_regres([-1,0,1],y4[1])
    MSE += mln4.bp_regres([0,0,0],y4[2])
    MSE += mln4.bp_regres([1,1,0],y4[3])
    MSE += mln4.bp_regres([-1,-1,-1],y4[4])
    MSE += mln4.bp_regres([-1,1,-1],y4[5])
    MSE = MSE / 6
    if(i%20 == 0):
        print ('MSE', i, '=',MSE)
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('predicted values:',y4)
print ()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.  1. -2.  1.] mrelu
Layer 1 Neuron 1 : [ 1. -2.  0.  1.] mrelu
Layer 2 Neuron 0 : [1. 0. 1.] sigmoid
Layer 3 Neuron 0 : [-1.  2.] linear



'original values:'

[1.0474258731775667,
 1.2709120776506935,
 1.1192029220221178,
 1.0474258731775667,
 1.0474258731775667,
 1.0066928509242847]


MSE 0 = 0.7888223085104592
MSE 20 = 0.03975597524662481
MSE 40 = 0.025690261520938344
MSE 60 = 0.004341713509646305
MSE 80 = 0.0006488928398983284
MSE 100 = 0.00019671343385545606
MSE 120 = 8.406219359129452e-05
MSE 140 = 5.4809343328723995e-05
MSE 160 = 4.4684051974188694e-05
MSE 180 = 3.9593192287750025e-05
MSE 200 = 3.6205651370519354e-05
MSE 220 = 3.3553440408558704e-05
MSE 240 = 3.127930079802735e-05
MSE 260 = 2.922564631837677e-05
MSE 280 = 2.731695545728887e-05
MSE 300 = 2.551693478307936e-05
MSE 320 = 2.3809191702296966e-05
MSE 340 = 2.2187282755614983e-05
MSE 360 = 2.0649318746640405e-05
MSE 380 = 1.9195055609863053e-05

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.02973968  0.17036274 -1.09666959  0.39327675] mrelu
Layer 1 Neuron 1 : [-1.09996944 -0.0030099   0.07708762  0.96141911] mrelu
Layer 2 Neuron 0 : [-0.93070435  0.38298721 -0.65325135] sigmoid
Layer 3 Neuron 0 : [0.74467469 0.86734761] linear



'predicted values:'

[1.0495491096433924,
 1.2716323205730653,
 1.1225846336300946,
 1.0463390431718171,
 1.0467116730981936,
 1.0068560712321397]

# Example 7 -- Regres class. עובד באופן עקרוני
# relu בעייתי

In [71]:
reg1=Regres(layers=[3,2,1,1], eta=0.3)
reg1.set_weights([[[1,1,0,0],[0,0,0,0]],[[0,0,0]],[[0,0]]])
X=[
[2,1,0],
[-1,0,1],
[0,0,0],
[1,1,0],
[-1,-1,-1],
[-1,1,-1],
]
y=[1.0474258731775667,
 1.2689414213699952,
 1.1192029220221178,
 1.0474258731775667,
 1.0474258731775667,
 1.0066928509242847]
reg1.set_hidden_activ('relu')
reg1.regres_network.network[2][0].set_activ('sigmoid') # changing specific activ func
reg1.fit(X,y, epochs=10)
reg1.printWeights()
pred=reg1.predict(X)
print (pred)
print()
# display ('weight_history',reg1.weight_history)
display ('MSE',reg1.MSE_history)

Model fitted.
self.weight_history - list of lists of weights propagation
self.MSE_history - list of MSEs propagation

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 1.00040383e+00  1.00019067e+00 -7.61928188e-04  4.62352645e-08] sigmoid
Layer 1 Neuron 1 : [-1.33988799e-04 -9.45825661e-05  1.21994284e-04 -1.69855173e-04] sigmoid
Layer 2 Neuron 0 : [-0.04399879 -0.0014526  -0.00291742] sigmoid
Layer 3 Neuron 0 : [0.40616469 0.81726402] linear

[1.0157211942002111, 1.0187758858446403, 1.017742685157543, 1.0160416343083993, 1.0194437487138914, 1.0177420753656825]



'MSE'

[0.8936673506717919,
 0.049136682384134815,
 0.049006395767603156,
 0.04889689821951907,
 0.04878857321336636,
 0.04868136549301467,
 0.048575225318883725,
 0.048470104973456656,
 0.04836595867604385,
 0.048262742502818735]

# Example -- Regres_scale. צריך חשוב ולהשלים

In [74]:
# nuzhno podumat' kak vesa obratno perevesti. mozhet inverse_transform
reg2=Regres_scale(layers=[3,2,1,1], eta=0.1)
reg2.set_weights([[[1,1,0,0],[0,0,0,0]],[[0,0,0]],[[0,0]]])
X=[
[2,1,0],
[-1,0,1],
[0,0,0],
[1,1,0],
[-1,-1,-1],
[-1,1,-1],
]
y=[1.0474258731775667,
 1.2689414213699952,
 1.1192029220221178,
 1.0474258731775667,
 1.0474258731775667,
 1.0066928509242847]
reg2.set_hidden_activ('relu')
reg2.regres_network.network[2][0].set_activ('sigmoid') # changing specific activ func
reg2.fit(X,y,epochs=200)
reg2.printWeights()
pred=reg2.predict(X)
pred


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.9517462   1.06201428 -0.06407585  0.06933407] relu
Layer 1 Neuron 1 : [0. 0. 0. 0.] relu
Layer 2 Neuron 0 : [-0.20894407  0.          0.00917428] sigmoid
Layer 3 Neuron 0 : [0.46600036 0.84566849] linear



[1.0081438427807037,
 1.0797374688505368,
 1.078049747612554,
 1.029768336904302,
 1.0797374688505368,
 1.073806571646052]

# Example 6 -- https://mmuratarat.github.io/2020-01-09/backpropagation

In [24]:
# generating output values
mln6 = MultiLayerNeuron(layers=[5,3,1], eta=.5)
mln6.set_activ('sigmoid')
mln6.set_output_activ('linear')
mln6.set_weights([[[.19,.33,.4,.51,.54,.1],
                   [.55,.16,.35,.85,.49,.1],
                   [.76,.97,.7,.85,.57,.1]],
                  [[.1,.03,-.17,.1]]
                  ]) # setting initial weights
mln6.printWeights()

# training
for i in range(100):
    MSE = 0.0
    MSE += mln6.bp_regres([.5,.1,1,0,0],[0.1])
    MSE += mln6.bp_regres([.3,.2,0,1,0],[.6])
    MSE += mln6.bp_regres([.7,.9,0,0,1],[.4])
    MSE += mln6.bp_regres([.8,.1,1,0,0],[.1])
    MSE = MSE / 4
    if(i%10 == 0):
        print ('MSE', i, '=',MSE)

mln6.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0.19 0.33 0.4  0.51 0.54 0.1 ] sigmoid
Layer 1 Neuron 1 : [0.55 0.16 0.35 0.85 0.49 0.1 ] sigmoid
Layer 1 Neuron 2 : [0.76 0.97 0.7  0.85 0.57 0.1 ] sigmoid
Layer 2 Neuron 0 : [ 0.1   0.03 -0.17  0.1 ] linear

MSE 0 = 3.097759865123268
MSE 10 = 0.2948017967667316
MSE 20 = 0.11235016703947856
MSE 30 = 0.039031749833149484
MSE 40 = 0.005346702764168406
MSE 50 = 0.00047155150910413527
MSE 60 = 9.822149422722454e-05
MSE 70 = 6.0504726410544755e-05
MSE 80 = 4.67746394940529e-05
MSE 90 = 3.966114337742912e-05

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.92948745 -0.3432972  -0.4785581   0.21811547 -0.0456272  -1.65606983] sigmoid
Layer 1 Neuron 1 : [-1.01311017 -0.23274925 -1.57166574  1.18384536  0.1929425  -1.78487787] sigmoid
Layer 1 Neuron 2 : [-0.09277171 -0.3986799   1.19699659 -0.59778185 -0.68424799 -2.10503325] sigmoid
Layer 2 Neuron 0 : [ 0.25752661  0.97942477 -1.06640139  0.35525422] linear



# Example 8 -- Relu check

In [75]:
reg2=Regres(layers=[1,1], eta=0.1)
X=[[0],[1],[2],[3],[4],[5]]
y=[0,1,2,3,4,5]
reg2.printWeights()

reg2.set_hidden_activ('linear')
reg2.fit(X,y,epochs=5)
reg2.printWeights()
pred=reg2.predict(X)
print (pred)
print()
display ('weight_history',reg2.weight_history)
display ('MSE',reg2.MSE_history)


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.29512974 -0.53203254] linear

Model fitted.
self.weight_history - list of lists of weights propagation
self.MSE_history - list of MSEs propagation

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [1.00051536e+00 9.10473977e-04] linear

[0.0009104739771413302, 1.0014258366057147, 2.001941199234288, 3.0024565618628616, 4.002971924491435, 5.0034872871200085]



'weight_history'

[[array([1.00051536e+00, 9.10473977e-04])],
 [array([1.00051536e+00, 9.10473977e-04])],
 [array([1.00051536e+00, 9.10473977e-04])],
 [array([1.00051536e+00, 9.10473977e-04])],
 [array([1.00051536e+00, 9.10473977e-04])]]

'MSE'

[1.9344016814855012,
 0.004671844782619629,
 0.0006999578980332885,
 0.00010487100531295113,
 1.5712270389761933e-05]

# Example 9 -- mrelu check. working

In [29]:
mln4 = MultiLayerNeuron(layers=[1,2,1]) # generating MLN with random weights
mln4.set_activ('mrelu', param=0.1)
# mln4.set_activ('relu')
mln4.set_output_activ('linear')
mln4.set_weights([[[-1.1,1.74],[3.35,-6.44]],[[-1.41,2.05,2.2]]]) # setting weights
mln4.printWeights()

y4=[] # generating output values
y4.append(mln4.run([0])[0])
y4.append(mln4.run([1])[0])
y4.append(mln4.run([2])[0])
y4.append(mln4.run([3])[0])
y4.append(mln4.run([4])[0])
y4.append(mln4.run([5])[0])
display ('original values:',y4)
print ()

# training model and predicting values
mln4 = MultiLayerNeuron(layers=[1,2,1], eta=0.005) # generating MLN with random weights
mln4.set_activ('mrelu', param=0.1)
# mln4.set_activ('relu')
mln4.set_output_activ('linear')

for i in range(4000):
    MSE = 0.0
    MSE += mln4.bp_regres([0],y4[0])
    MSE += mln4.bp_regres([1],y4[1])
    MSE += mln4.bp_regres([2],y4[2])
    MSE += mln4.bp_regres([3],y4[3])
    MSE += mln4.bp_regres([4],y4[4])
    MSE += mln4.bp_regres([5],y4[5])
    MSE = MSE / 6
    if(i%200 == 0):
        print ('MSE', i, '=',MSE)
mln4.printWeights()

y4=[]
y4.append(mln4.run([0])[0])
y4.append(mln4.run([1])[0])
y4.append(mln4.run([2])[0])
y4.append(mln4.run([3])[0])
y4.append(mln4.run([4])[0])
y4.append(mln4.run([5])[0])
display ('predicted values:',y4)
print ()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-1.1   1.74] mrelu
Layer 1 Neuron 1 : [ 3.35 -6.44] mrelu
Layer 2 Neuron 0 : [-1.41  2.05  2.2 ] linear



'original values:'

[-1.5735999999999999,
 0.6641500000000005,
 2.7978599999999996,
 9.82046,
 16.843059999999998,
 23.865659999999995]


MSE 0 = 671.4250566100033
MSE 200 = 0.8092721640832977
MSE 400 = 0.7962073006169635
MSE 600 = 0.7971261494840092
MSE 800 = 0.7972330549338689
MSE 1000 = 0.7972497016603947
MSE 1200 = 0.7972580132928803
MSE 1400 = 0.7972655275375907
MSE 1600 = 0.7972729254463321
MSE 1800 = 0.7972802669972082
MSE 2000 = 0.797287555967234
MSE 2200 = 0.797294791047459
MSE 2400 = 0.797301970471412
MSE 2600 = 0.7973090924353562
MSE 2800 = 0.7973161551373282
MSE 3000 = 0.7973231567816467
MSE 3200 = 0.7973300955802879
MSE 3400 = 0.7973369697540064
MSE 3600 = 0.797343777533409
MSE 3800 = 0.7973505171600418

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 1.76992419 -2.81978615] mrelu
Layer 1 Neuron 1 : [ 0.99086476 -1.8767268 ] mrelu
Layer 2 Neuron 0 : [2.95506151 1.86081069 0.37839775] linear



'predicted values:'

[-0.804089729462234,
 -0.09668507001681798,
 2.7016161315653964,
 9.775662726019556,
 16.849709320473718,
 23.923755914927874]

In [27]:
reg2=Regres(layers=[1,2,1], eta=0.0001)
X=[[1],[2],[3],[4],[5]]
y=[1.,4.,9.,16.,25.]
# reg2.printWeights()

reg2.set_hidden_activ('mrelu', param=0.1)
reg2.fit(X,y,epochs=500)
reg2.printWeights()
pred=reg2.predict(X)
print (pred)
print()
# display ('weight_history',reg2.weight_history)
display ('MSE',np.array(reg2.MSE_history).min())


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 2.09091588 -0.66835824] mrelu
Layer 1 Neuron 1 : [-0.51182635 -0.27236682] mrelu
Layer 2 Neuron 0 : [ 2.18219708  0.87253538 -0.18489267] linear

[2.9194084620075587, 7.482198982337523, 12.044989502667487, 16.607780022997453, 21.17057054332742]



'MSE'

32.839772410041725